In [5]:
import pandas as pd
import numpy as np

In [6]:
patch = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
airport_raw = pd.read_csv(patch, sep = ',', engine = 'python',
           header = None,
           names = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'])
airport_raw = airport_raw.drop(airport_raw[airport_raw['IATA'] == '\\N'].index)
airport_raw.head()

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz database time zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [7]:
patch_1 = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat'
rout_raw = pd.read_csv(patch_1, sep = ',', engine = 'python',
           header = None,
           names = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment'])
rout_raw.head()

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
0,2B,410,AER,2965,KZN,2990,NaN,0,CR2
1,2B,410,ASF,2966,KZN,2990,NaN,0,CR2
2,2B,410,ASF,2966,MRV,2962,NaN,0,CR2
3,2B,410,CEK,2968,KZN,2990,NaN,0,CR2
4,2B,410,CEK,2968,OVB,4078,NaN,0,CR2


In [8]:
rout_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat', sep = ',', engine = 'python',
           header = None,
           names = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport', 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment'])

airport_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', sep = ',', engine = 'python',
           header = None,
           names = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz database time zone', 'Type', 'Source'])
airport_raw = airport_raw[airport_raw['IATA'] != '\\N']


df_1 = pd.DataFrame(rout_raw['Destination airport'].value_counts())
df_1.reset_index(inplace=True)
df_1.columns = ['Airport', 'Прилёты']

df_2 = pd.DataFrame(rout_raw['Source airport'].value_counts())
df_2.reset_index(inplace=True)
df_2.columns = ['Airport', 'Вылеты']

itog = pd.merge(df_1, df_2, on='Airport', how='outer').fillna(0)
itog['Total'] = itog['Прилёты'] + itog['Вылеты']
result = pd.merge(itog, airport_raw[['IATA', 'Country', 'Name']], left_on='Airport', right_on='IATA', how='left')
result = result.drop(columns=['IATA'])  # удаляем дублирующийся столбец


pivot = result.pivot_table(index=['Country', 'Name'], values=['Вылеты', 'Прилёты', 'Total'])
pivot = pivot.sort_values('Total', ascending=False).head(10)
pivot = pivot.astype(int)

pivot = pivot.reindex(columns=['Вылеты', 'Прилёты', 'Total'])
pivot

Вылеты  \
Country        Name                                                       
United States  Hartsfield Jackson Atlanta International Airport     915   
               Chicago O'Hare International Airport                 558   
China          Beijing Capital International Airport                535   
United Kingdom London Heathrow Airport                              527   
France         Charles de Gaulle International Airport              524   
United States  Los Angeles International Airport                    492   
Germany        Frankfurt am Main Airport                            497   
United States  Dallas Fort Worth International Airport              469   
               John F Kennedy International Airport                 456   
Netherlands    Amsterdam Airport Schiphol                           453   

                                                                 Прилёты  \
Country        Name                                                        
United States  Hartsfield Jackson Atlanta International Airport      911   
               Chicago O'Hare International Airport                  550   
China          Beijing Capital International Airport                 534   
United Kingdom London Heathrow Airport                               524   
France         Charles de Gaulle International Airport               517   
United States  Los Angeles International Airport                     498   
Germany        Frankfurt am Main Airport                             493   
United States  Dallas Fort Worth International Airport               467   
               John F Kennedy International Airport                  455   
Netherlands    Amsterdam Airport Schiphol                            450   

                                                                 Total  
Country        Name                                                     
United States  Hartsfield Jackson Atlanta International Airport   1826  
               Chicago O'Hare International Airport               1108  
China          Beijing Capital International Airport              1069  
United Kingdom London Heathrow Airport                            1051  
France         Charles de Gaulle International Airport            1041  
United States  Los Angeles International Airport                   990  
Germany        Frankfurt am Main Airport                           990  
United States  Dallas Fort Worth International Airport             936  
               John F Kennedy International Airport                911  
Netherlands    Amsterdam Airport Schiphol                          903